In [3]:
import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from ultralytics import YOLO

In [4]:
# Constants
TARGET_SIZE = 512  # Output image size

def remove_padding_and_resize(img, target_size=TARGET_SIZE):
    """画像のリサイズと中央配置を行う関数
    
    与えられた画像をアスペクト比を維持しながら指定サイズにリサイズし、
    黒色の背景の中央に配置します。
    
    Args:
        img: 入力画像（numpy配列）
        target_size: 出力画像の目標サイズ（デフォルト値：TARGET_SIZE）
        
    Returns:
        tuple: (処理後の画像, (x開始位置, y開始位置, スケール))
            - 処理後の画像: target_size × target_sizeの正方形画像
            - x開始位置: リサイズ後の画像の配置開始x座標
            - y開始位置: リサイズ後の画像の配置開始y座標
            - スケール: 元画像に対する縮小/拡大率
    """
    # 入力画像が3チャンネルでない場合（マスク画像など）、適切に処理
    is_grayscale = len(img.shape) == 2
    
    h, w = img.shape[:2]
    scale = target_size / max(h, w)
    new_w = int(w * scale)
    new_h = int(h * scale)
    
    # グレースケール画像とカラー画像でリサイズ方法を分ける
    if is_grayscale:
        resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_NEAREST)
        final_img = np.zeros((target_size, target_size), dtype=img.dtype)
    else:
        resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        final_img = np.zeros((target_size, target_size, 3), dtype=img.dtype)
    
    start_x = (target_size - new_w) // 2
    start_y = (target_size - new_h) // 2
    
    if is_grayscale:
        final_img[start_y:start_y+new_h, start_x:start_x+new_w] = resized
    else:
        final_img[start_y:start_y+new_h, start_x:start_x+new_w] = resized
    
    return final_img, (start_x, start_y, scale)

def process_dataset(model_path, input_dir, output_dir, conf_threshold=0.9):
    """CT画像とセグメンテーションマスクを処理するメイン関数
    
    Args:
        model_path: YOLOモデルのパス
        input_dir: 入力データセットディレクトリ
        output_dir: 出力ディレクトリ
        conf_threshold: 検出信頼度の閾値（デフォルト：0.25）
    """
    # 入力ディレクトリのパス
    images_dir = os.path.join(input_dir, "imagesTr") # ここを場合に応じてimagesTrなどに変更
    labels_dir = os.path.join(input_dir, "labelsTr")
    
    # 出力ディレクトリの作成
    output_images_dir = os.path.join(output_dir, "imagesTr")
    output_labels_dir = os.path.join(output_dir, "labelsTr")
    
    os.makedirs(output_images_dir, exist_ok=True)
    os.makedirs(output_labels_dir, exist_ok=True)
    
    # YOLOモデルのロード
    model = YOLO(model_path)
    
    # 画像ファイルの一覧を取得
    image_files = [f for f in os.listdir(images_dir) if f.endswith('.png')]
    
    successful_crops = 0
    failed_crops = 0
    
    for img_file in tqdm(image_files, desc="Processing images"):
        img_path = os.path.join(images_dir, img_file)
        
        # 画像の読み込み
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Could not read image {img_path}")
            continue
            
        # 対応するマスクファイル名を取得
        # 例: 6375522_20_R_FLOOR-3_0000.png → 6375522_20_R_FLOOR-3.png
        mask_file = img_file.rsplit('_', 1)[0] + '.png'
        mask_path = os.path.join(labels_dir, mask_file)
        
        # マスクが存在するか確認
        if not os.path.exists(mask_path):
            print(f"Warning: Mask file not found: {mask_path}")
            continue
            
        # マスクの読み込み
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            print(f"Warning: Could not read mask {mask_path}")
            continue
        
        # YOLOでROIを検出
        results = model.predict(
            source=img,
            conf=conf_threshold,
            verbose=False
        )
        
        roi_detected = False
        
        for result in results:
            boxes = result.boxes
            if len(boxes) > 0:
                # 最も確信度の高いROIを使用
                best_box = boxes[boxes.conf.argmax()]
                roi_detected = True
                
                # ROIの座標を取得
                x1, y1, x2, y2 = map(int, best_box.xyxy.cpu().numpy()[0])
                
                # 画像とマスクからROIをクロップ
                cropped_img = img[y1:y2, x1:x2]
                cropped_mask = mask[y1:y2, x1:x2]
                
                # カラー画像をグレースケールに変換
                cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
                
                # クロップした画像とマスクをリサイズして中央配置
                processed_img, _ = remove_padding_and_resize(cropped_img, target_size=TARGET_SIZE)
                processed_mask, _ = remove_padding_and_resize(cropped_mask, target_size=TARGET_SIZE)
                
                # 画像とマスクを保存
                cv2.imwrite(os.path.join(output_images_dir, img_file), processed_img)
                cv2.imwrite(os.path.join(output_labels_dir, mask_file), processed_mask)
                
                successful_crops += 1
                break  # 最も確信度の高いROIのみを処理
        
        if not roi_detected:
            print(f"Warning: No ROI detected in {img_file}")
            failed_crops += 1
    
    # 処理結果の統計を表示
    total_images = len(image_files)
    print(f"\nProcessing Summary:")
    print(f"Total images processed: {total_images}")
    print(f"ROI successfully detected: {successful_crops} ({successful_crops/total_images*100:.2f}%)")
    print(f"ROI detection failed: {failed_crops} ({failed_crops/total_images*100:.2f}%)")

# メイン実行部分
if __name__ == "__main__":
    # パスの設定
    INPUT_DIR = "/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset006_Orbital_Trap_All"
    OUTPUT_DIR = "/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset007_Orbital_Trap_All_ROI"
    MODEL_PATH = "/Users/yuma/Yuma-Kanematsu/nnUNet/src/yolov8n_ROI_241108.pt"  # YOLOモデルのパスを指定
    
    # メイン処理を実行
    process_dataset(MODEL_PATH, INPUT_DIR, OUTPUT_DIR, conf_threshold=0.9)

Processing images:  58%|█████▊    | 302/518 [00:12<00:08, 24.30it/s]

Processing images: 100%|██████████| 518/518 [00:20<00:00, 25.16it/s]


Processing Summary:
Total images processed: 518
ROI successfully detected: 517 (99.81%)
ROI detection failed: 1 (0.19%)
